<a href="https://colab.research.google.com/github/premdub/AutoML/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install tpot mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### 2...Whether WEATHER condition predict the SHOW or NO SHOW

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML


In [3]:
import pandas as pd
weather = pd.read_csv('/content/combined_df_2.csv')
weather

,Unnamed: 0,patient_id,appointment_date,appointment_date_time,appointment_start_time,appointment_duration,appointment_yosi_noshow1,date,formatted date,Summary,Precip Type,Humidity
0,0,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 12:14,Partly Cloudy,rain,0.89
1,1,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 10:45,Partly Cloudy,rain,0.62
2,2,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 16:20,Partly Cloudy,rain,0.68
3,3,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 9:25,Partly Cloudy,rain,0.82
4,4,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 12:50,Partly Cloudy,rain,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...
21025,21025,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 10:30,Mostly Cloudy,rain,0.62
21026,21026,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 10:00,Mostly Cloudy,rain,0.61
21027,21027,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 11:39,Mostly Cloudy,rain,0.49
21028,21028,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 18:00,Mostly Cloudy,rain,0.58


In [4]:
weather.columns

Index(['Unnamed: 0', 'patient_id', 'appointment_date', 'appointment_date_time',
       'appointment_start_time', 'appointment_duration',
       'appointment_yosi_noshow1', 'date', 'formatted date', 'Summary',
       'Precip Type', 'Humidity'],
      dtype='object')

Potential variables of interest


1.   Precip Type (categorical)
2.   Humidity (continuous)
3.   appointment_duration (continuous)




In [7]:
weather['Precip Type'].describe()

count     21030
unique        3
top        rain
freq      18163
Name: Precip Type, dtype: object

In [8]:
weather['Humidity'].describe()

count    21029.000000
mean         0.740204
std          0.191387
min          0.130000
25%          0.620000
50%          0.790000
75%          0.900000
max          1.000000
Name: Humidity, dtype: float64

In [9]:
weather['appointment_duration'].value_counts()

10     7593
5      6899
15     2609
20     2340
30      499
40      498
60      285
120     205
45       68
90       22
255       7
150       5
Name: appointment_duration, dtype: int64

In [18]:
weather['Humidity'] = pd.to_numeric(weather['Humidity'], errors='coerce')
weather['weather_SON'] = weather['Humidity'].apply(lambda x: 'bad weather' if x >0.5  else 'nice weather')
weather.drop('Humidity', axis=1, inplace=True)
weather['weather_SON'].value_counts()


bad weather     17911
nice weather     3119
Name: weather_SON, dtype: int64

In [19]:
X = weather.drop(columns=['weather_SON'])

In [31]:
y = weather["weather_SON"]

In [32]:
X

,Unnamed: 0,patient_id,appointment_date,appointment_date_time,appointment_start_time,appointment_duration,appointment_yosi_noshow1,date,formatted date,Summary
0,0,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 12:14,Partly Cloudy
1,1,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 10:45,Partly Cloudy
2,2,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 16:20,Partly Cloudy
3,3,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 9:25,Partly Cloudy
4,4,357685,4/14/2021,4/14/2021 12:14,12:14:00,10,N,4/14/2021,4/14/2021 12:50,Partly Cloudy
...,...,...,...,...,...,...,...,...,...,...
21025,21025,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 10:30,Mostly Cloudy
21026,21026,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 10:00,Mostly Cloudy
21027,21027,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 11:39,Mostly Cloudy
21028,21028,34447,4/10/2020,4/10/2020 8:07,8:07:00,10,Y,4/10/2020,4/10/2020 18:00,Mostly Cloudy


In [33]:
y

0         bad weather
1         bad weather
2         bad weather
3         bad weather
4         bad weather
             ...     
21025     bad weather
21026     bad weather
21027    nice weather
21028     bad weather
21029    nice weather
Name: weather_SON, Length: 21030, dtype: object

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25)

In [38]:
X_test

,Unnamed: 0,patient_id,appointment_date,appointment_date_time,appointment_start_time,appointment_duration,appointment_yosi_noshow1,date,formatted date,Summary
14537,14537,178187,11/9/2021,11/9/2021 7:50,7:50:00,20,Y,11/9/2021,11/9/2021 14:40,Foggy
1577,1577,424902,9/5/2021,9/5/2021 15:36,15:36:00,10,Y,9/5/2021,9/5/2021 10:05,Mostly Cloudy
3837,3837,575865,12/11/2020,12/11/2020 10:00,10:00:00,15,N,12/11/2020,12/11/2020 11:05,Partly Cloudy
20404,20404,97420,10/28/2021,10/28/2021 15:10,15:10:00,10,N,10/28/2021,10/28/2021 13:20,Clear
15778,15778,37700,9/16/2021,9/16/2021 15:00,15:00:00,10,Y,9/16/2021,9/16/2021 14:50,Overcast
...,...,...,...,...,...,...,...,...,...,...
3713,3713,139499,8/9/2021,8/9/2021 10:00,10:00:00,5,N,8/9/2021,8/9/2021 18:38,Clear
297,297,380381,9/17/2021,9/17/2021 16:40,16:40:00,10,N,9/17/2021,9/17/2021 18:35,Partly Cloudy
8539,8539,302711,1/27/2021,1/27/2021 17:05,17:05:00,10,Y,1/27/2021,1/27/2021 16:25,Clear
14556,14556,99061,5/17/2021,5/17/2021 13:30,13:30:00,30,Y,5/17/2021,5/17/2021 9:00,Partly Cloudy


In [39]:
automl = AutoML(results_path="weather_binary_los", mode="Explain")

In [40]:
automl.fit(X_train, y_train)  

Linear algorithm was disabled.
AutoML directory: weather_binary_los
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline logloss 0.419861 trained in 0.73 seconds
There was an error during 2_DecisionTree training.
Please check weather_binary_los/errors.md for details.
* Step default_algorithms will try to check up to 3 models
There was an error during 2_Default_Xgboost training.
Please check weather_binary_los/errors.md for details.
There was an error during 3_Default_NeuralNetwork training.
Please check weather_binary_los/errors.md for details.
There was an error during 4_Default_RandomForest training.
Please check weather_binary_los/errors.md for details.
* Step ensemble will try to check up to 1 

AutoML(results_path='weather_binary_los')

In [41]:
pred = automl.predict(X_test)
pred

array(['nice weather', 'nice weather', 'nice weather', ...,
       'nice weather', 'nice weather', 'nice weather'], dtype=object)

In [42]:
automl.report()

In [43]:
# load in the data model 

automl_HAI_los = AutoML(results_path="HAI_binary_los")

In [45]:
# create a new dataset that follows the same data structure as the training set
X_withlos = weather.sample(25)
X_withoutlos = X_withlos.drop(columns=['weather_SON'])

In [46]:
X_withlos

,Unnamed: 0,patient_id,appointment_date,appointment_date_time,appointment_start_time,appointment_duration,appointment_yosi_noshow1,date,formatted date,Summary,weather_SON
4288,4288,97512,10/26/2019,10/26/2019 9:35,9:35:00,5,Y,10/26/2019,10/26/2019 11:45,Overcast,bad weather
8834,8834,284997,1/4/2021,1/4/2021 8:15,8:15:00,5,Y,1/4/2021,1/4/2021 13:15,Mostly Cloudy,bad weather
6121,6121,247465,8/23/2021,8/23/2021 8:40,8:40:00,5,Y,8/23/2021,8/23/2021 14:00,Mostly Cloudy,bad weather
3845,3845,575865,12/11/2020,12/11/2020 10:00,10:00:00,15,N,12/11/2020,12/11/2020 14:10,Clear,bad weather
17463,17463,50003,12/18/2020,12/18/2020 17:20,17:20:00,10,Y,12/18/2020,12/18/2020 15:10,Mostly Cloudy,bad weather
1359,1359,396751,11/11/2020,11/11/2020 15:30,15:30:00,30,Y,11/11/2020,11/11/2020 12:00,Partly Cloudy,bad weather
5425,5425,96595,4/20/2021,4/20/2021 15:10,15:10:00,20,N,4/20/2021,4/20/2021 10:50,Partly Cloudy,bad weather
16566,16566,406937,8/7/2021,8/7/2021 13:50,13:50:00,5,Y,8/7/2021,8/7/2021 11:15,Overcast,bad weather
7663,7663,287716,1/7/2021,1/7/2021 12:40,12:40:00,5,Y,1/7/2021,1/7/2021 13:50,Partly Cloudy,bad weather
16535,16535,277860,4/30/2021,4/30/2021 9:55,9:55:00,10,Y,4/30/2021,4/30/2021 9:55,Partly Cloudy,bad weather


In [47]:
X_withoutlos

,Unnamed: 0,patient_id,appointment_date,appointment_date_time,appointment_start_time,appointment_duration,appointment_yosi_noshow1,date,formatted date,Summary
4288,4288,97512,10/26/2019,10/26/2019 9:35,9:35:00,5,Y,10/26/2019,10/26/2019 11:45,Overcast
8834,8834,284997,1/4/2021,1/4/2021 8:15,8:15:00,5,Y,1/4/2021,1/4/2021 13:15,Mostly Cloudy
6121,6121,247465,8/23/2021,8/23/2021 8:40,8:40:00,5,Y,8/23/2021,8/23/2021 14:00,Mostly Cloudy
3845,3845,575865,12/11/2020,12/11/2020 10:00,10:00:00,15,N,12/11/2020,12/11/2020 14:10,Clear
17463,17463,50003,12/18/2020,12/18/2020 17:20,17:20:00,10,Y,12/18/2020,12/18/2020 15:10,Mostly Cloudy
1359,1359,396751,11/11/2020,11/11/2020 15:30,15:30:00,30,Y,11/11/2020,11/11/2020 12:00,Partly Cloudy
5425,5425,96595,4/20/2021,4/20/2021 15:10,15:10:00,20,N,4/20/2021,4/20/2021 10:50,Partly Cloudy
16566,16566,406937,8/7/2021,8/7/2021 13:50,13:50:00,5,Y,8/7/2021,8/7/2021 11:15,Overcast
7663,7663,287716,1/7/2021,1/7/2021 12:40,12:40:00,5,Y,1/7/2021,1/7/2021 13:50,Partly Cloudy
16535,16535,277860,4/30/2021,4/30/2021 9:55,9:55:00,10,Y,4/30/2021,4/30/2021 9:55,Partly Cloudy


In [48]:
predict = automl.predict(X_withoutlos)
predict

array(['nice weather', 'nice weather', 'nice weather', 'nice weather',
       'nice weather', 'nice weather', 'nice weather', 'nice weather',
       'nice weather', 'nice weather', 'nice weather', 'nice weather',
       'nice weather', 'nice weather', 'nice weather', 'nice weather',
       'nice weather', 'nice weather', 'nice weather', 'nice weather',
       'nice weather', 'nice weather', 'nice weather', 'nice weather',
       'nice weather'], dtype=object)

In [49]:
# actual values from X_withlos
values_actual = X_withlos['weather_SON'].values.tolist()
values_predicted = predict.tolist()
output = pd.DataFrame({'actual': values_actual, 'predicted': values_predicted})
output

,actual,predicted
0,bad weather,nice weather
1,bad weather,nice weather
2,bad weather,nice weather
3,bad weather,nice weather
4,bad weather,nice weather
5,bad weather,nice weather
6,bad weather,nice weather
7,bad weather,nice weather
8,bad weather,nice weather
9,bad weather,nice weather


In [50]:
# get current working directory
import os
os.getcwd()

'/content'

In [51]:
folders = os.listdir()
foldersML = [x for x in folders if x.startswith('weather')]
print(foldersML)

['weather_binary_los']


In [52]:
!zip -r /content/weather_binary_los.zip.zip /content/weather_binary_los

  adding: content/weather_binary_los/ (stored 0%)
  adding: content/weather_binary_los/params.json (deflated 60%)
  adding: content/weather_binary_los/errors.md (deflated 83%)
  adding: content/weather_binary_los/data_info.json (deflated 78%)
  adding: content/weather_binary_los/README.html (deflated 30%)
  adding: content/weather_binary_los/README.md (deflated 53%)
  adding: content/weather_binary_los/2_Default_Xgboost/ (stored 0%)
  adding: content/weather_binary_los/4_Default_RandomForest/ (stored 0%)
  adding: content/weather_binary_los/progress.json (deflated 90%)
  adding: content/weather_binary_los/2_DecisionTree/ (stored 0%)
  adding: content/weather_binary_los/leaderboard.csv (deflated 20%)
  adding: content/weather_binary_los/split_validation_indices.npy (deflated 67%)
  adding: content/weather_binary_los/split_train_indices.npy (deflated 67%)
  adding: content/weather_binary_los/1_Baseline/ (stored 0%)
  adding: content/weather_binary_los/1_Baseline/cumulative_gains_curve.pn